In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pmdarima
from matplotlib.dates import MONDAY
from matplotlib.dates import WeekdayLocator
import statsmodels.tsa.arima.model as stm
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import ssl
from nelson_siegel_svensson import NelsonSiegelSvenssonCurve
from nelson_siegel_svensson.calibrate import calibrate_ns_ols
from nelson_siegel_svensson.calibrate import calibrate_nss_ols
import arch
import warnings

warnings.filterwarnings('ignore')

In [2]:
url_trates = "https://www.treasury.gov/resource-center/data-chart-center/interest-rates/Pages/TextView.aspx?data=yieldYear&year=2019"
url_gold_etf = "https://finance.yahoo.com/quote/GLD/history?period1=1569888000&period2=1575072000&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true"
url_equity_etf = "https://finance.yahoo.com/quote/CSUK.L/history?period1=1569888000&period2=1575072000&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true"

In [3]:
#Creating the interest rate data frame
dfs = pd.read_html(url_trates, header = 0, index_col=0, parse_dates = True)
df_trate = dfs[1]
df_trate = df_trate.loc['2019-10':'2019-11'].iloc[:,5:]

In [5]:
#Creating the gold ETF data frame
df_goldetf = pd.read_html(url_gold_etf, header = 0, index_col=0, parse_dates = True)[0].iloc[:-1]
df_goldetf.index = pd.to_datetime(df_goldetf.index)
df_goldetf = df_goldetf.sort_index(ascending = True)
df_goldetf = df_goldetf.astype(float)

HTTPError: HTTP Error 404: Not Found

In [ ]:
#Creating the equity ETF dataframe
df_equityetf = pd.read_html(url_equity_etf, header = 0, index_col=0)[0].iloc[:-1]
df_equityetf.index = pd.to_datetime(df_equityetf.index)
df_equityetf = df_equityetf.sort_index(ascending = True)
df_equityetf = df_equityetf.astype(float)

In [ ]:
#Daily return on gold etf and equity etf
df_goldetf_daily_return = np.log(df_goldetf['Adj Close**']/df_goldetf['Adj Close**'].shift(1)).iloc[1:]
df_equityetf_daily_return = np.log(df_equityetf['Adj Close**']/df_equityetf['Adj Close**'].shift(1)).iloc[1:]

In [ ]:
#Average return for yield for benchmark Security
df_avgmonthlyyield = df_trate.resample('M').mean()
#Average price of gold ETF on a monthly basis
df_avggoldETF = df_goldetf['Adj Close**'].resample('M').mean()
#Average price of equityETF on a monthly basis
df_avgequityETF = df_equityetf['Adj Close**'].resample('M').mean()

In [ ]:
#Standard Deviation for yield of benchmark security
df_tratestd = df_trate.resample('M').std()
#Standard Deviation of gold ETF on a monthly basis
df_goldETFstd = df_goldetf['Adj Close**'].resample('M').std()
#Average price of equityETF on a monthly basis
df_equityETFstd = df_equityetf['Adj Close**'].resample('M').std()

In [ ]:
#Plotting the yields
mondays = WeekdayLocator(MONDAY)
fig, ax = plt.subplots(figsize=(10,6))
ax.set_title("Interest rate chart for Treasury Bond - Oct and Nov '19")
ax.plot(df_trate)
ax.set_xlabel("Trade Date")
plt.xticks(rotation=30)
ax.xaxis.set_major_locator(mondays)
ax.set_ylabel("Interest Rate (in % terms)")
ax.legend(df_trate.columns.values)
plt.show()

In [ ]:
#Plotting the gold and equity ETF 
#mondays = WeekdayLocator(MONDAY)
fig, ax1 = plt.subplots(figsize=(10,6))
ax1.set_title("Gold ETF and Equity ETF - Oct and Nov '19")
lns1 = ax1.plot(df_goldetf['Adj Close**'], color = 'gold', label = "Gold ETF")
ax1.set_xlabel("Trade Date")
plt.xticks(rotation=30)
ax1.xaxis.set_major_locator(mondays)
ax1.set_ylabel("Gold ETF price in $")
ax2 = ax1.twinx()
ax2.set_ylabel("MSCI500 UK - Equity ETF price in $")
lns2 = ax2.plot(df_equityetf['Adj Close**'], color = 'blue', label = "Equity ETF")
lns = lns1+lns2
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs)
plt.show()

In [ ]:
ssl._create_default_https_context = ssl._create_unverified_context

#US Treasury Yield Data Frame
dfs = pd.read_html(url_trates, header = 0, index_col=0, parse_dates = True)
df_trate1 = dfs[1]
df_trate1 = df_trate1.loc['2019-10':'2019-11'].iloc[:,5:]
df_trate1 = df_trate1/100

#Nelson-Siegel model to fit the daily October and November 2019 yield curves
t = np.array([2, 3, 5, 7, 10, 20, 30])
yields = df_trate1.to_numpy()
df_nsyields = pd.DataFrame(columns = ['beta0', 'beta1', 'beta2', 'tau'])

for y in yields:
    curve1, status1 = calibrate_ns_ols(t, y)
    assert status1.success

    ns_elements = str(curve1).split(',')
    ns_beta0 = float(ns_elements[0].split('=')[1])
    ns_beta1 = float(ns_elements[1].split('=')[1])
    ns_beta2 = float(ns_elements[2].split('=')[1])
    ns_tau = float(ns_elements[3].split('=')[1][0:2])

    df_nsyields = df_nsyields.append({'beta0' : ns_beta0, 'beta1' : ns_beta1, 'beta2' : ns_beta2, 'tau' : ns_tau}, ignore_index = True)

print('Fit Based on Nelson-Siegel Model for Daily October and November 2019 Yield Curves')
print(df_nsyields)
print(" ")
print("Based on the charts shown in the attached Excel file (Yield Curve Fitting Results Approach 1.xlsx):\nThe level parameter (beta0) registers an increase in October 2019 from 0.022 to 0.023. It then rises to 0.025 in November before declining to roughly where it was at October end.\nThe slope parameter (beta1) declines from 0.0015 to 0.0014 in October, registers a further decline in November to a low of -0.0027 before rising to -0.0009 by November end.\n The curvature parameter (beta2) remains more or less stable in October within the band of -0.032 to -0.028. It then steadily increases in November to -0.024")
print(" ")


#Nelson-Siegel-Svensson model to fit the daily October and November 2019 yield curves
df_nssyields = pd.DataFrame(columns = ['beta0', 'beta1', 'beta2', 'beta3', 'tau1', 'tau2'])

for y in yields:
    curve2, status2 = calibrate_nss_ols(t, y)
    assert status2.success

    nss_elements = str(curve2).split(',')
    nss_beta0 = float(nss_elements[0].split('=')[1])
    nss_beta1 = float(nss_elements[1].split('=')[1])
    nss_beta2 = float(nss_elements[2].split('=')[1])
    nss_beta3 = float(nss_elements[3].split('=')[1])
    nss_tau1 = float(nss_elements[4].split('=')[1])
    nss_tau2 = float(nss_elements[5].split('=')[1][0:2])

    df_nssyields = df_nssyields.append({'beta0' : nss_beta0, 'beta1' : nss_beta1, 'beta2' : nss_beta2, 'beta3' : nss_beta3, 'tau1' : nss_tau1, 'tau2' : nss_tau2}, ignore_index = True)

print('Fit Based on Nelson-Siegel-Svensson Model for Daily October and November 2019 Yield Curves')
print(df_nssyields)

In [ ]:
#Test for stationarity of the time series. We notice that the time series is non-stationary using ADF test
adfuller(df_goldetf_daily_return['Oct-2019'])

In [ ]:
#We differentiate the gold ETF daily returns and thus obtain a stationary series
d1_gold_etf_returns = df_goldetf_daily_return.diff().dropna()
#Proof to display stationarity of the Time Series - The first order differencing is causing the time series of gold ETF to be stationary
#Refer adf p-value for stationarity
adfuller(d1_gold_etf_returns['Oct-2019'])

In [ ]:
plot_pacf(df_goldetf_daily_return['Oct-2019'],lags = 8);
plot_acf(df_goldetf_daily_return['Oct-2019'], lags = 8);
arimaoctgoldetfreturn = ARIMA(df_goldetf_daily_return['Oct-2019'], order = (1,0,0))
arimaoctgoldetfreturnfit = arimaoctgoldetfreturn.fit()
arimaoctgoldetfreturnfit.summary()

In [ ]:
print('Standard deviation of the residuals is :',arimaoctgoldetfreturnfit.resid.std())
arimaoctgoldetfreturnfit.resid.plot(kind='kde', title = 'Verifying normality of the residuals');

In [ ]:
#We notice that the time series ARIMA(0,1,1)has the lowest AIC at -145 and the p-value of the MA co-efficient is less than the critical value.
#when we tried with a ARIMA(1,1,1) model, we had AIC of -143.15 and the p-value of the AR coefficient was 0.722 which is more than the critical value

In [ ]:
#Proof to display stationarity of the Time Series - The 2nd order differencing is causing the time series of gold ETF to be stationary
#Refer adf p-value for stationarity
adfuller(df_goldetf_daily_return.diff().diff().dropna()['Nov-2019'])

plot_pacf(df_goldetf_daily_return.diff().diff().dropna()['Nov-2019'],lags = 8);
plot_acf(df_goldetf_daily_return.diff().diff().dropna()['Nov-2019'], lags = 8);
  
arimanovgoldetfreturn = ARIMA(df_goldetf_daily_return['Nov-2019'], order = (1,1,0))
arimanovgoldetfreturnfit = arimanovgoldetfreturn.fit()
arimanovgoldetfreturnfit.summary()
#We notice that the time series has the lowest AIC at -132.4 and the p-value of the AR co-efficient is less than the critical value.
#when we tried with a ARIMA(1,1,1) model, we had AIC of -129.9 and the p-value of the AR and MA were 0.394 and 0.539 which is more than the critical value

In [ ]:
#Measuring the Equity ETF ARIMA
adfuller(df_equityetf_daily_return['Oct-2019'])
#We notice that the time series is stationary and the p-value is much less than the critical value which indicates that we reject the null hypothesis and thus the data is stationary

In [ ]:
from pandas.plotting import autocorrelation_plot
autocorrelation_plot(df_equityetf_daily_return['Oct-2019'])
plot_pacf(df_equityetf_daily_return['Oct-2019'],lags = 8);
plot_acf(df_equityetf_daily_return['Oct-2019'], lags = 10);
arimaoctequityreturn = ARIMA(df_equityetf_daily_return['Oct-2019'], order = (0,0,1))
arimaoctequityreturnfit = arimaoctequityreturn.fit()
arimaoctequityreturnfit.summary()

In [ ]:
print('Standard deviation of the residuals is :',arimaoctequityreturnfit.resid.std())
arimaoctequityreturnfit.resid.plot(kind = 'kde', title = 'Verifying the normality of residuals');

In [ ]:
#Measuring the Equity ETF ARIMA stationarity
adfuller(df_equityetf_daily_return['Nov-2019'])
#We notice that the time series is stationary and the p-value is much less than the critical value which indicates that we reject the null hypothesis and thus the data is stationary

In [ ]:
adfuller(df_equityetf_daily_return['Nov-2019'].diff().dropna())

In [ ]:
autocorrelation_plot(df_equityetf_daily_return['Nov-2019'], label = 'Auto-correlation for Nov Equity ETF')
#autocorrelation_plot(df_equityetf_daily_return['Nov-2019'].diff().dropna(), label = 'Auto-correlation for Nov Equity ETF - 1st order differencing')
plot_pacf(df_equityetf_daily_return['Nov-2019'],lags = 8);
plot_acf(df_equityetf_daily_return['Nov-2019'], lags = 10);
arimanovequityreturn = ARIMA(df_equityetf_daily_return['Nov-2019'], order = (0,0,1))
arimanovequityreturnfit = arimanovequityreturn.fit()
arimanovequityreturnfit.summary()

In [ ]:
print('Standard deviation of the residuals is :',arimanovequityreturnfit.resid.std())
arimanovequityreturnfit.resid.plot(kind = 'kde', title = 'Verifying the normality of residuals');

In [ ]:
df_dailyhighlowgoldetf = df_goldetf['High']-df_goldetf['Low']
'''Using gold ETF prices, find the daily high minus low for each month.  
Compute the average for October.  
Compute the average for November.'''
df_dailyhighlowgoldetf['Oct-2019']

In [ ]:
df_dailyhighlowgoldetf['Nov-2019']

In [ ]:
#Using the gold ETF returns, find the standard deviation for October.  Repeat for November
df_goldetfreturnsstddev = df_goldetf_daily_return.resample('M').std()
df_goldetfreturnsstddev.index = ['October', 'November']
df_goldetfreturnsstddev

In [ ]:
'''Using equity ETF prices, find the daily high minus low for each month.  
Compute the average for October.  Compute the average for November.'''
df_dailyhighlowequityetf = df_equityetf['High']-df_equityetf['Low']

In [ ]:
df_dailyhighlowequityetf['Oct-2019']

In [ ]:
df_dailyhighlowequityetf['Nov-2019']

In [ ]:
#Average of difference between High-Low for each month
df_highlowgoldequitymean = df_dailyhighlowequityetf.resample('M').mean()
df_highlowgoldequitymean.index = ['October', 'November']
df_highlowgoldequitymean

In [ ]:
#Using equity ETF returns, find the standard deviation for October.  Repeat for November
df_equityetfreturnsstddev = df_equityetf_daily_return.resample('M').std()
df_equityetfreturnsstddev.index = ['October', 'November']
df_equityetfreturnsstddev

In [ ]:
#GARCH(1,1) Model on the Residuals of the ARIMA(1,0,0) model - October 2019 Gold ETF
print("GARCH(1,1) Model on the Residuals of the ARIMA(1,0,0) model - October 2019 Gold ETF")
print(" ")
garch_goldetf_oct = arch.arch_model(arimaoctgoldetfreturnfit.resid, p=1, q=1)
garch_goldetf_oct_fit = garch_goldetf_oct.fit()
print(garch_goldetf_oct_fit.summary())

In [ ]:
# GARCH(1,1) Model on the Residuals of the ARIMA(1,1,0) model - November 2019 Gold ETF
print("GARCH(1,1) Model on the Residuals of the ARIMA(1,1,0) model - November 2019 Gold ETF")
print(" ")
garch_goldetf_nov = arch.arch_model(arimanovgoldetfreturnfit.resid, p=1, q=1)
garch_goldetf_nov_fit = garch_goldetf_nov.fit()
print(garch_goldetf_nov_fit.summary())
print(" ")
print("Between October and November, for the volatility model, alpha[1] parameter increases, beta[1] parameter declines and the 95% CI for both parameters is wider for November.")

In [ ]:
#GARCH(1,1) Model on the Residuals of the ARIMA(0,0,1) model - October 2019 Equity ETF
print("GARCH(1,1) Model on the Residuals of the ARIMA(0,0,1) model - October 2019 Equity ETF")
print(" ")
garch_equityetf_oct = arch.arch_model(arimaoctequityreturnfit.resid, p=1, q=1)
garch_equityetf_oct_fit = garch_equityetf_oct.fit()
print(garch_equityetf_oct_fit.summary())

In [ ]:
#GARCH(1,1) Model on the Residuals of the ARIMA(0,0,1) model - November 2019 Equity ETF
print("GARCH(1,1) Model on the Residuals of the ARIMA(0,0,1) model - November 2019 Equity ETF")
print(" ")

garch_equityetf_nov = arch.arch_model(arimanovequityreturnfit.resid, p=1, q=1)
garch_equityetf_nov_fit = garch_equityetf_nov.fit()
print(garch_equityetf_nov_fit.summary())
print(" ")
print("Between October and November, for the volatility model, alpha[1] parameter declines, beta[1] parameter remains more or less stable and the 95% CI for both parameters is wider for November.")

In [ ]:
print('The pearsons correlation coefficient between Gold and Equity ETF for the month of Oct-2019 is:',np.corrcoef(df_goldetf_daily_return['Oct-2019'], df_equityetf_daily_return['Oct-2019'])[1][0])

In [ ]:
print('The pearsons correlation coefficient between Gold and Equity ETF for the month of Nov-2019 is:',pd.DataFrame(pd.concat([df_equityetf_daily_return['Nov-2019'], df_goldetf_daily_return['Nov-2019']], axis=1)).dropna().corr(method = 'pearson').iloc[0,1])

In [ ]:
df_all = pd.concat([df_equityetf['Adj Close**'], df_goldetf['Adj Close**'], df_trate/100, df_equityetf_daily_return, df_goldetf_daily_return],join = 'inner', axis=1)
df_all.columns = ['Equity ETF','Gold ETF','Equity ETF Return','Gold ETF Return','2 yr','3 yr', '5 yr','7 yr', '10 yr','20 yr','30 yr']
df_all['Oct-2019'].corr(method = 'pearson')

import seaborn as sns
fig, ax = plt.subplots(figsize=(10,10)) 
sns.heatmap(df_all['Oct-2019'].corr(method = 'pearson'), linewidths=0.1, cmap = 'YlOrBr', annot=True);

In [ ]:
fig, ax = plt.subplots(figsize=(10,10)) 
sns.heatmap(df_all['Nov-2019'].corr(method = 'pearson'), linewidths=0.1, cmap = 'YlOrBr', annot=True);


In [ ]:
#Equally weighted lag - 7 days correlation between Equity ETF with yields
corr_equity_etf = df_all[['Equity ETF','2 yr','3 yr', '5 yr','7 yr', '10 yr']].rolling(7).corr().dropna().xs('Equity ETF', level=1).iloc[:,1:]
corr_equity_etf = pd.DataFrame(corr_equity_etf)

In [ ]:
#Plotting the equity ETF and the T-rates
mondays = WeekdayLocator(MONDAY)
fig, ax1 = plt.subplots(figsize=(10,6))
ax1.set_title("Equity ETF prices and its correlation (7days lag) with Treasury rates")
lns1 = ax1.plot(df_all['Equity ETF'], color = 'Blue', label = "Equity ETF")
ax1.set_xlabel("Trade Date")
plt.xticks(rotation=30)
ax1.xaxis.set_major_locator(mondays)
ax1.set_ylabel("Equity ETF price in $")
ax2 = ax1.twinx()
ax2.set_ylabel("Correlation of Equity ETF with treasury rates")

labels=['2 yr','3 yr', '5 yr','7 yr', '10 yr']
i=0
for label in labels:
    ax2.plot(corr_equity_etf.iloc[:,i], label=label)
    i+=1
ax1.legend()
ax2.legend()
plt.show()

In [ ]:
#Equally weighted lag - 7 dayscorrelation between Equity ETF with yields
corr_gold_etf = df_all[['Gold ETF','2 yr','3 yr', '5 yr','7 yr', '10 yr']].rolling(7).corr().dropna().xs('Gold ETF', level=1).iloc[:,1:]
corr_gold_etf = pd.DataFrame(corr_gold_etf)

In [ ]:
#Plotting the Gold ETF and the T-rates
mondays = WeekdayLocator(MONDAY)
fig, ax1 = plt.subplots(figsize=(10,6))
ax1.set_title("Gold ETF prices and its correlation (7-days lag) with Treasury rates")
lns1 = ax1.plot(df_all['Gold ETF'], color = 'Blue', label = "Gold ETF")
ax1.set_xlabel("Trade Date")
plt.xticks(rotation=30)
ax1.xaxis.set_major_locator(mondays)
ax1.set_ylabel("Gold ETF price in $")
ax2 = ax1.twinx()
ax2.set_ylabel("Correlation of Gold ETF with treasury rates")

labels=['2 yr','3 yr', '5 yr','7 yr', '10 yr']
i=0
for label in labels:
    ax2.plot(corr_gold_etf.iloc[:,i], label=label)
    i+=1
ax1.legend(loc = 6)
ax2.legend()
plt.show()